**Задание**

Разобраться с моделькой генерации текста, собрать самим или взять датасет с вебинара и обучить генератор текстов

In [1]:
import numpy as np
import tensorflow as tf
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
#загружаем файл
text = open('dostoevsky.txt', 'rb').read().decode(encoding='utf-8')

# Определяем длину текста (количество букв в нем)
print('Length of text: {} characters'.format(len(text)))

Length of text: 1280652 characters


In [3]:
print(text[:500])

Тут на горе паслось большое стадо свиней, и они просили Его, чтобы позволил им войти в них. Он позволил им. Бесы, вышедши из человека, вошли в свиней; и бросилось стадо с крутизны в озеро и потонуло. Пастухи, увидя случившееся, побежали и рассказали в городе и по деревням. И вышли жители смотреть случившееся и, пришедши к Иисусу, нашли человека, из которого вышли бесы, сидящего у ног Иисусовых, одетого и в здравом уме, и ужаснулись. Видевшие же рассказали им, как исцелился бесновавшийся.

Еванге


In [4]:
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

150 unique characters


In [5]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [6]:
print(text[:200]), print(text_as_int[:200])

Тут на горе паслось большое стадо свиней, и они просили Его, чтобы позволил им войти в них. Он позволил им. Бесы, вышедши из человека, вошли в свиней; и бросилось стадо с крутизны в озеро и потонуло. 
[ 99 129 128   1 123 110   1 113 124 126 115   1 125 110 127 121 124 127
 138   1 111 124 121 138 134 124 115   1 127 128 110 114 124   1 127 112
 118 123 115 119   6   1 118   1 124 123 118   1 125 126 124 127 118 121
 118   1  87 113 124   6   1 133 128 124 111 137   1 125 124 117 112 124
 121 118 121   1 118 122   1 112 124 119 128 118   1 112   1 123 118 131
   8   1  95 123   1 125 124 117 112 124 121 118 121   1 118 122   8   1
  83 115 127 137   6   1 112 137 134 115 114 134 118   1 118 117   1 133
 115 121 124 112 115 120 110   6   1 112 124 134 121 118   1 112   1 127
 112 118 123 115 119  20   1 118   1 111 126 124 127 118 121 124 127 138
   1 127 128 110 114 124   1 127   1 120 126 129 128 118 117 123 137   1
 112   1 124 117 115 126 124   1 118   1 125 124 128 124 123 129 121 

(None, None)

In [7]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(10):
    print(idx2char[i.numpy()])

Т
у
т
 
н
а
 
г
о
р


In [8]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Тут на горе паслось большое стадо свиней, и они просили Его, чтобы позволил им войти в них. Он позвол'
'ил им. Бесы, вышедши из человека, вошли в свиней; и бросилось стадо с крутизны в озеро и потонуло. Па'
'стухи, увидя случившееся, побежали и рассказали в городе и по деревням. И вышли жители смотреть случи'
'вшееся и, пришедши к Иисусу, нашли человека, из которого вышли бесы, сидящего у ног Иисусовых, одетог'
'о и в здравом уме, и ужаснулись. Видевшие же рассказали им, как исцелился бесновавшийся.\n\nЕвангелие о'


In [9]:
# Разбиваем каждый батч на признаки и целевую переменную (последнюю букву)
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

#применяем функцию split_input_target ко всем батчам
dataset = sequences.map(split_input_target)

In [10]:
for input_example, target_example in dataset:
    print(f'Количество батчей: {len(dataset)}')
    print(f'Данные: {input_example}')
    print(f'Целевая переменная: {target_example}')
    break

Количество батчей: 12679
Данные: [ 99 129 128   1 123 110   1 113 124 126 115   1 125 110 127 121 124 127
 138   1 111 124 121 138 134 124 115   1 127 128 110 114 124   1 127 112
 118 123 115 119   6   1 118   1 124 123 118   1 125 126 124 127 118 121
 118   1  87 113 124   6   1 133 128 124 111 137   1 125 124 117 112 124
 121 118 121   1 118 122   1 112 124 119 128 118   1 112   1 123 118 131
   8   1  95 123   1 125 124 117 112 124]
Целевая переменная: [129 128   1 123 110   1 113 124 126 115   1 125 110 127 121 124 127 138
   1 111 124 121 138 134 124 115   1 127 128 110 114 124   1 127 112 118
 123 115 119   6   1 118   1 124 123 118   1 125 126 124 127 118 121 118
   1  87 113 124   6   1 133 128 124 111 137   1 125 124 117 112 124 121
 118 121   1 118 122   1 112 124 119 128 118   1 112   1 123 118 131   8
   1  95 123   1 125 124 117 112 124 121]


In [11]:
for input_example, target_example in dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Тут на горе паслось большое стадо свиней, и они просили Его, чтобы позволил им войти в них. Он позво'
Target data: 'ут на горе паслось большое стадо свиней, и они просили Его, чтобы позволил им войти в них. Он позвол'


In [12]:
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [13]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [14]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [15]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [16]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 150) # (batch_size, sequence_length, vocab_size)


In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           38400     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 lstm_1 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 lstm_2 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 lstm_3 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 dense (Dense)               (64, None, 150)           153750    
                                                                 
Total params: 30,617,238
Trainable params: 30,617,238
No

In [18]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1) #количество независимых выборок 1

#убираем лишнюю размерность (список индексов)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

array([126, 129, 132,  77,  41, 143,  31,  48,  36,  89,  95,  85,  17,
        67,  78,  49,  11, 119,  46, 138, 145, 122,  19,  39,   3,  82,
        71,  79,  85, 106, 100, 127, 119,  99,  28, 132,  28,  49,  45,
        12, 108,  69, 146,  14, 149,  84,  57,  75, 141,   0, 111,  40,
        73, 141,  11,  38,  82, 132,  44,  13,  77,  27,  32,  86,  73,
        12,  95,   6, 135,  37,  26,   6,  35, 127, 131,  38, 149,  31,
        24, 111,  27,  48,  48,  42,  24,  60,   5,   4, 137,  10,  27,
       128,  98,  74, 129,  98,  67, 116,  64, 122])

In [19]:
#входной пример
#возвращаем строку
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()

#предсказанная буква
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'л рядом с помещиком.\n\n– Еще ему сахару! – приказал Семен Яковлевич, когда уже налили стакан; положил'

Next Char Predictions: 
 'руцçT–JbOЗОГ8uèc2й]ь’м:R(АzéГЬУсйТGцGc[3Юx“5…Вkáя\nбS»я2QАцZ4çFKД»3О,щPE,NсхQ…JCбFbbVCn*)ы1FтСàуСuжrм'


In [20]:
#функция для просчета loss
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 150)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       5.0103226


In [21]:
model.compile(optimizer='adam', loss=loss)

In [24]:
# сохраняем чекпоинты
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=100*5,
    save_weights_only=True,
    )

In [26]:
#обучаем модель

num_epoch = 20
history = model.fit(dataset, epochs=num_epoch, callbacks=[checkpoint_callback])

Epoch 1/20
198/198 [==============================] - 71s 351ms/step - loss: 1.2283
Epoch 2/20
198/198 [==============================] - 72s 359ms/step - loss: 1.1716
Epoch 3/20
198/198 [==============================] - 71s 353ms/step - loss: 1.1133
Epoch 4/20
198/198 [==============================] - 71s 353ms/step - loss: 1.0546
Epoch 5/20
198/198 [==============================] - 72s 359ms/step - loss: 0.9961
Epoch 6/20
198/198 [==============================] - 71s 354ms/step - loss: 0.9331
Epoch 7/20
198/198 [==============================] - 71s 355ms/step - loss: 0.8697
Epoch 8/20
198/198 [==============================] - 71s 351ms/step - loss: 0.8040
Epoch 9/20
198/198 [==============================] - 71s 354ms/step - loss: 0.7424
Epoch 10/20
198/198 [==============================] - 72s 357ms/step - loss: 0.6830
Epoch 11/20
198/198 [==============================] - 70s 349ms/step - loss: 0.6271
Epoch 12/20
198/198 [==============================] - 72s 358ms/step - lo

In [27]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_20'

In [28]:
#строим модель
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [29]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 256)            38400     
                                                                 
 lstm_4 (LSTM)               (1, None, 1024)           5246976   
                                                                 
 lstm_5 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 lstm_6 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 lstm_7 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 dense_1 (Dense)             (1, None, 150)            153750    
                                                                 
Total params: 30,617,238
Trainable params: 30,617,238


In [35]:
def generate_text(model, start_string):
  
    num_generate = 3000
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []

    temperature = 0.03

    model.reset_states()
    for i in range(num_generate):
        
        predictions = model(input_eval)
       
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))


In [36]:
text_ = generate_text(model, start_string=u"На следующий день они ")
print(text_)

На следующий день они за своею полною подписью. Каков!

– Хуже всего то, что вы вывертываетесь. Многие повскочили с мест, иные с ним познакомился.

– Это всё равно, – пролепетал Петр Степанович, как бы совсем не заметив мгновение за собой». Прибавлю, однако ж, что, кроме моего разрешения общественной формулы, насилу на стол и стал пред ним и, прежде всякого слова, пристально вгляделся в его лицо.

– Ого уж так и будет. А старикашка согласится мигом. Каждый должен был бы ее принять на себя всю жизнь никуда не сойду. Хотите, встаньте и воды, – проговорила madame Виргинская.

– Совершенно ничего. Ваша просьба исполняется, и вы навсегда устранили и теперь в его время, как и я сам.

– Ну так знайте, что Шатов считает этот донос своих грязные свои острова и позволяет себе фамьли Степану Трофимовичу ее напечатать, за совершенною ее, в наше время, невинностью, но все почему-то старые посещения от Ставрогина была и поговорила с ним какой-то необычный шум шагов и голосов, подобный давешний испуг

===============================================

Так же была попытка сделать модель на GRU слоях, но модель постоянно переобучалась и не удалось подобрать параметры для генерации хоть сколько нибудь осмысленного текста.